In [2]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [28]:
df_csv = pd.read_csv('datasets/reddit_dataset.csv',error_bad_lines=False)
# df_csv = df_csv.dropna()

In [29]:
df_csv.head(10)

,reddit_comment,label
0,a subsection of retarded hungarians? ohh boy. ...,1
1,hiii. just got off work. 444 is mainly the typ...,0
2,wow i guess soyboys are the same in every country,0
3,owen benjamin's soyboy song goes for every cou...,0
4,"> ""y'all hear sumn?"" by all means i live in a ...",0
5,[removed],0
6,ah a liberal has slipped in. you can tell beca...,1
7,wouldn't the defenders or whatever they are as...,0
8,'inclusive' = not white,0
9,“harvard is working to be more inclusive.” “oh...,0


In [31]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=df_csv['reddit_comment']
comments_attack['label']=df_csv['label']
dataframe = comments_attack

In [32]:
# Text preprocessing
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
# # dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('"',' ',x)))
# # dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('”',' ',x)))


In [33]:
len(dataframe['label'])

21654

In [34]:
print(dataframe['comment'])

0        a subsection of retarded hungarians? ohh boy. ...
1        hiii. just got off work. 444 is mainly the typ...
2        wow i guess soyboys are the same in every country
3        owen benjamin's soyboy song goes for every cou...
4        > "y'all hear sumn?" by all means i live in a ...
                               ...                        
21649    op, stop being a faggot and post videos next t...
21650    in this 20 minute long video, top hate and cha...
21651    no clue whos these e-celebs are, but at this p...
21652        i didn’t insult you, why would you insult me?
21653                        because you are living a lie.
Name: comment, Length: 21654, dtype: object


In [35]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.2, random_state=42)

In [36]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  17323
test comments length:  4331


In [37]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)



C:\Users\mjahan18\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [38]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [39]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, WordLevel TF-IDF:   accuracy: 0.8007388593858231     f1 score: 0.7436882574633419
NB, N-Gram Vectors:   accuracy: 0.7788039713691988     f1 score: 0.7053905944352409
NB, CharLevel Vectors:   accuracy: 0.8166705148926345   f1 score: 0.7744026345826864


In [40]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

LR, WordLevel TF-IDF:   accuracy: 0.8757792657584853   f1 score: 0.8647465524316914
LR, N-Gram Vectors:   accuracy: 0.7799584391595474   f1 score: 0.7048298994362617
LR, CharLevel Vectors:   accuracy: 0.8815516047102285   f1 score: 0.8732140583903649
